In [1]:
import pyltr
import numpy as np
import sklearn.tree
import random
import math
import os

In [23]:
NUM_TREES = 50
DATA_FOLDER = 'data/'
EXPERIMENTS = ['long-tail','noisy','query-length/c1','query-length/c2','query-length/c3','']
NUM_EXPERIMENTS = 5
MODES = ['train','val','test']



In [24]:
file_dict = {
    'long-tail' :{
        'train': 'train_longtail_suggest_matrix.npz',
        'val': 'val_longtail_suggest_matrix.npz',
        'test': 'test_longtail_suggest_matrix.npz',
    },
    'noisy' :{
        'train': 'tr_noisy_suggest_matrix.npz',
        'val': 'val_noisy_suggest_matrix.npz',
        'test': 'test_noisy_suggest_matrix.npz',
    },
    'query-length/c1' : {
        'train': 'tr_c1_suggest_matrix.npz',
        'val': 'val_c1_suggest_matrix.npz',
        'test': 'test_c1_suggest_matrix.npz',
    },
    'query-length/c2' : {
        'train': 'tr_c2_suggest_matrix.npz',
        'val': 'val_c2_suggest_matrix.npz',
        'test': 'test_c2_suggest_matrix.npz',
    },
    'query-length/c3': {
        'train': 'tr_c3_suggest_matrix.npz',
        'val': 'val_c3_suggest_matrix.npz',
        'test': 'test_c3_suggest_matrix.npz',
    },
    '' : {
        'train': 'train_suggest_matrix.npz',
        'val': 'val_suggest_matrix.npz',
        'test': 'test_suggest_matrix.npz',
    }
}

#col0=anchor ID, col1=suggestion ID. col 2-19 features, col 19 is scores
def make_sessions_ids(anchor_ids):
    
    sessions_ids = np.zeros([len(anchor_ids)])
    iters = len(anchor_ids) /20
    for i in range(iters):
        sessions_ids[i*20:(i+1)*20] = i
    return sessions_ids

def make_data(data_folder, experiment, file_dict,mode ,feature_idx=20):
            
        path = os.path.join(data_folder, experiment, file_dict[experiment][mode] )
        
        
        data_train = np.load(path)['arr_0']
       
        data_anchor_ids = data_train[:,0]
        data_suggestion_ids = data_train[:,1]
        data_features = data_train[:,3:feature_idx]
        data_label_scores = data_train[:,20]
        data_session_ids = make_sessions_ids(data_anchor_ids)
        
        return data_anchor_ids, data_suggestion_ids, data_features, data_label_scores, data_session_ids
    
def MMR_Score(Epred, test_data_session_ids, test_data_label_scores):
    sessions = len(test_data_session_ids)/20
    sum_score = 0
    for session_idx in range(0, sessions):
        pred_idx = np.argmax(Epred[session_idx*20:(session_idx +1)*20])
        sum_score += 1. /( pred_idx + 1)
    return sum_score/ sessions


In [6]:
"""
Some small demo's
metric = pyltr.metrics.NDCG(k=20)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=n_trees,
    learning_rate=1./n_trees,
    max_features=0.3,
    query_subsample=(1./n_trees),
    max_leaf_nodes=10,
    min_samples_leaf=200,
    verbose=1,
    random_state=None
)           

monitor = pyltr.models.monitors.ValidationMonitor(
    val_data_features, val_data_label_scores, val_data_session_ids, metric=metric, stop_after=5)
    
    model.fit(train_data_features, train_data_label_scores, train_data_session_ids)
    
    Epred = model.predict(test_data_features)

print 'Random ranking:', metric.calc_mean_random(val_data_session_ids, val_data_label_scores)
print 'Our model:', metric.calc_mean(test_data_session_ids, test_data_label_scores, Epred)

print MMR_Score(Epred, test_data_session_ids, test_data_label_scores)

"""



In [ ]:
metric = pyltr.metrics.NDCG(k=20)
result_dict = {
    'long-tail' :[],
    'noisy' :[],
    'query-length/c1' : [],
    'query-length/c2' : [],
    'query-length/c3': [],
    '' : []
}

for experiment in EXPERIMENTS:
    for epoch in range(NUM_EXPERIMENTS):
        print "--- new experiment ---" 
        train_data_anchor_ids, train_data_suggestion_ids, train_data_features, \
        train_data_label_scores, train_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'train' ,feature_idx=20)
        val_data_anchor_ids, val_data_suggestion_ids, val_data_features, \
        val_data_label_scores, val_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'val' ,feature_idx=20) 

        test_data_anchor_ids, test_data_suggestion_ids, test_data_features, \
        test_data_label_scores, test_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'test' ,feature_idx=20) 
        
        model = pyltr.models.LambdaMART(
            metric=metric,
            n_estimators=NUM_TREES,
            learning_rate=1./NUM_TREES,
            max_features=0.3,
            query_subsample=(1./NUM_TREES),
            max_leaf_nodes=10,
            min_samples_leaf=200,
            verbose=0,
            random_state=None
        )
        
        model.fit(train_data_features, train_data_label_scores, train_data_session_ids)
        
        Epred = model.predict(test_data_features)
        
        random_ranking = metric.calc_mean_random(val_data_session_ids, val_data_label_scores)
        model_ndcg = metric.calc_mean(test_data_session_ids, test_data_label_scores, Epred)
        mrr_score = MMR_Score(Epred, test_data_session_ids, test_data_label_scores)
        print "Experiment: ", epoch + 1
        print "MMR Score for " + experiment + " : ", mrr_score
        result_dict[experiment].append(mrr_score)
    print "__________________________________________________"

--- new experiment ---
Experiment:  0
MMR Score for long-tail :  0.347198452573
--- new experiment ---
